In [2]:
import pandas as pd
from sqlalchemy import create_engine

In [3]:
city_file = "resources/cities.csv"
# Read our transit data into pandas
city_df = pd.read_csv(city_file)
city_df.head()

,id,name,coords,start_year,url_name,country,country_state
0,5,Aberdeen,POINT(-2.15 57.15),2017,aberdeen,Scotland,NaN
1,6,Adelaide,POINT(138.6 -34.91666667),2017,adelaide,Australia,NaN
2,7,Algiers,POINT(3 36.83333333),2017,algiers,Algeria,NaN
3,9,Ankara,POINT(32.91666667 39.91666667),2017,ankara,Turkey,NaN
4,16,Belém,POINT(-48.48333333 -1.466666667),2017,belem,Brazil,NaN


In [4]:
city_file ="resources/cities.csv"
city_df = pd.read_csv(city_file)
city_df.head()

,id,name,coords,start_year,url_name,country,country_state
0,5,Aberdeen,POINT(-2.15 57.15),2017,aberdeen,Scotland,NaN
1,6,Adelaide,POINT(138.6 -34.91666667),2017,adelaide,Australia,NaN
2,7,Algiers,POINT(3 36.83333333),2017,algiers,Algeria,NaN
3,9,Ankara,POINT(32.91666667 39.91666667),2017,ankara,Turkey,NaN
4,16,Belém,POINT(-48.48333333 -1.466666667),2017,belem,Brazil,NaN


In [5]:
station_file = "resources/stations.csv"
station_df = pd.read_csv(station_file)
station_df.head()

,id,name,geometry,buildstart,opening,closure,city_id
0,7694,Keisei Tsudanuma,POINT(140.024812197129 35.6837744784723),1921.0,1921.0,999999.0,114
1,6003,Kossuth Lajos tér,POINT(19.0462376564033 47.5054880717671),0.0,0.0,999999.0,29
2,7732,Saint-Charles,POINT(5.3801556 43.3024646),1973.0,1977.0,999999.0,74
3,7695,Keisei Makuhari-Hongo,POINT(140.042146725175 35.6726021159981),1991.0,1991.0,999999.0,114
4,7726,Chartreux,POINT(5.4014815 43.309129),1973.0,1977.0,999999.0,74


In [6]:
# Create a filtered dataframe from specific columns
city_cols = ["id", "name", "start_year","country"]
city_transformed= city_df[city_cols].copy()

# Rename the column headers
city_transformed = city_transformed.rename(columns={"id": "City_ID",
                                                      "name": "City_Name",
                                                      "start_year": "Start_Year",
                                                      "country":"Country"})

# Clean the data by dropping duplicates and setting the index
city_transformed.drop_duplicates("City_ID", inplace=True)
city_transformed.set_index("City_ID", inplace=True)

city_transformed.head()

,City_Name,Start_Year,Country
City_ID,,,
5,Aberdeen,2017,Scotland
6,Adelaide,2017,Australia
7,Algiers,2017,Algeria
9,Ankara,2017,Turkey
16,Belém,2017,Brazil


In [7]:
city_transformed.count()

City_Name     337
Start_Year    337
Country       337
dtype: int64

In [8]:
cities_dropped = city_transformed.dropna()
cities_dropped.count()

City_Name     337
Start_Year    337
Country       337
dtype: int64

In [9]:
station_cols = ["id", "name", "buildstart", "opening","city_id"]
station_rename = station_df[station_cols].copy()

# Rename the column headers
station_rename =station_rename.rename(columns={"id": "Station_ID",
                                                         "name": "Station_Name",
                                                         "buildstart": "Buildstart",
                                                         "opening": "Opening",
                                                        "city_id": "City_ID"})




In [10]:
# Set index
station_transformed = station_rename.copy()


In [11]:

station_transformed.set_index("Station_ID", inplace=True)

station_transformed.head()

,Station_Name,Buildstart,Opening,City_ID
Station_ID,,,,
7694,Keisei Tsudanuma,1921.0,1921.0,114
6003,Kossuth Lajos tér,0.0,0.0,29
7732,Saint-Charles,1973.0,1977.0,74
7695,Keisei Makuhari-Hongo,1991.0,1991.0,114
7726,Chartreux,1973.0,1977.0,74


In [12]:
station_transformed.count()

Station_Name    14624
Buildstart      14719
Opening         16102
City_ID         16171
dtype: int64

In [13]:
station_dropped = station_transformed.dropna()
station_dropped.count()

Station_Name    13125
Buildstart      13125
Opening         13125
City_ID         13125
dtype: int64

In [14]:
# Create a filtered dataframe from specific columns
master_df = pd.merge(city_transformed, station_rename, on='City_ID')


master_df.set_index("Station_ID", inplace=True)
master_df.head()

,City_ID,City_Name,Start_Year,Country,Station_Name,Buildstart,Opening
Station_ID,,,,,,,
8421,211,Ottawa,2000,Canada,Carling,2000.0,2001.0
8422,211,Ottawa,2000,Canada,Bayview,2000.0,2001.0
6324,147,Chicago,1892,United States,Kedzie,0.0,1907.0
6317,147,Chicago,1892,United States,Irving Park,0.0,1907.0
6368,147,Chicago,1892,United States,Grand,0.0,1921.0


In [15]:
rds_connection_string = "postgres:Rob11sts@localhost:5432/transit_systems"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [17]:
engine.table_names()

['cities', 'stations', 'master']

In [19]:
station_transformed.columns

Index(['Station_Name', 'Buildstart', 'Opening', 'City_ID'], dtype='object')

In [16]:
#load dataframe into Database-cities

city_transformed.to_sql(name='cities', con=engine, if_exists='append', index=True)


In [17]:
#load dataframe into Database-stations

station_transformed.to_sql(name='stations', con=engine, if_exists='append', index=True)



In [24]:
#load dataframe into Database-stations

master_df.to_sql(name='master', con=engine, if_exists='append', index=True)